In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import os
from matplotlib.cm import get_cmap
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [2]:
def plot_trend_labels(df, label_columns):
    """
    Function to plot trend labels and save the plot as an image.

    Parameters:
    df (pd.DataFrame): DataFrame containing 'Price' column and trend label columns.
    label_columns (list): List of column names containing trend labels.
    """
    
    # Define the slice size
    slice_size = 30
    
    # Ensure the DataFrame is large enough
    if len(df) <= slice_size:
        raise ValueError("DataFrame is too small for the specified slice size.")
    
    # Randomly select a slice that is not at the beginning or the end
    start_idx = np.random.randint(1, len(df) - slice_size - 1)
    df_slice = df.iloc[start_idx:start_idx + slice_size]

    # Identify the columns
    price_col = 'Price'
    
    time = df_slice.index.to_numpy()
    price = df_slice[price_col].to_numpy()

    # Generate a list of distinct colors
    color_map = cm.get_cmap('tab20', len(label_columns))
    colors = [color_map(i) for i in range(len(label_columns))]

    # Create subplots
    fig, axes = plt.subplots(len(label_columns), 1, figsize=(12, 4 * len(label_columns)), sharex=True)
    
    if len(label_columns) == 1:
        axes = [axes]  # Ensure axes is always a list of axes

    for i, label_col in enumerate(label_columns):
        labels = df_slice[label_col].to_numpy()
        min_price = min(price)
        max_price = max(price)
        y_limits = (min_price - 0.5, max_price + 0.5)

        axes[i].plot(time, price, label='Asset price', color='black', linewidth=2)
        axes[i].scatter(time, price, color='black', s=50, zorder=5)  # Big dots for price points
        axes[i].scatter(time, labels * (max_price + 1), color=colors[i], label='Trend labels', zorder=5, s=100)
        axes[i].plot(time, labels * (max_price + 1), color=colors[i], linewidth=2, zorder=4)
        axes[i].set_title(f'{label_col} Labeling', fontsize=16)
        axes[i].set_ylabel('Price', fontsize=14)
        axes[i].set_ylim(y_limits)

        # Set secondary y-axis for trend labels (-1, 0, 1)
        secax = axes[i].twinx()
        secax.set_ylim(-1.5, 1.5)
        secax.set_yticks([-1, 0, 1])
        secax.set_ylabel('Labels', fontsize=14)

        # Set primary y-axis to show the starting price and lowest price
        axes[i].set_yticks([min_price, max_price])
        axes[i].legend()

    # Common X label
    plt.xlabel('Number of Periods', fontsize=14)
    plt.xticks(rotation=45)
    plt.tight_layout(pad=2.0)

    # Create directory if it does not exist
    save_path = 'visualizations/trend_labeling.png'
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    # Save the plot
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()

In [3]:
def visualize_overall_returns_old(result_df, path):
    """
    Visualize the overall returns with and without transaction costs for each labeling method.
    
    Parameters:
    result_df (pd.DataFrame): The DataFrame containing 'Labeling method', 'Overall return', and 'Overall return with TC'.
    """
    # Define the full English names for the labeling methods
    labeling_method_names = {
        'excess_over_mean': 'Excess over Mean',
        'excess_over_median': 'Excess over Median',
        'fixed_time_horizon': 'Fixed Time Horizon',
        'triple_barrier': 'Triple Barrier',
        'tail_sets': 'Tail Sets',
        'matrix_flag': 'Matrix Flag',
        'trend_scanning': 'Trend Scanning',
        'buy_and_hold': 'Buy and Hold',
        "next_period": 'Next Period Labeling'
    }
    
    # Set up the bar positions
    labels = [labeling_method_names.get(label, label) for label in result_df['Labeling method']]
    overall_returns = result_df['Overall return']
    overall_returns_tc = result_df['Overall return with TC']
    
    x = range(len(labels))
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Define colors
    pink_color = '#DA70D6'
    purple_color = '#9932CC'
    black_color = '#000000'
    
    # Create the bars
    bar_width = 0.35
    ax.bar(x, overall_returns * 100, width=bar_width, color=pink_color, label='Accumulated return')
    ax.bar([p + bar_width for p in x], overall_returns_tc * 100, width=bar_width, color=purple_color, label='Accumulated return after transaction costs')
    
    # Add the labels, title and legend
    ax.set_xlabel('Labeling Method')
    ax.set_ylabel('Return in percentage of investment')
    ax.set_title('Overall Returns with and without Transaction Costs')
    ax.set_xticks([p + bar_width/2 for p in x])
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.legend()
    
    # Add a horizontal line at the 0 return mark
    ax.axhline(y=0, color=black_color, linewidth=1)
    
    # Display the plot
    #plt.tight_layout()
    #plt.show()
    
    # Create directory if it does not exist
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.savefig(path, bbox_inches='tight')

In [4]:
def visualize_overall_returns(result_df, path):
    """
    Visualize the overall returns with and without transaction costs for each labeling method.
    
    Parameters:
    result_df (pd.DataFrame): The DataFrame containing 'labeling_method', 'overall_return', and 'overall_return_TC'.
    path (str): The path to save the plot.
    """
    # Define the full English names for the labeling methods
    labeling_method_names = {
        'excess_over_mean': 'Excess over Mean',
        'excess_over_median': 'Excess over Median',
        'fixed_time_horizon': 'Fixed Time Horizon',
        'triple_barrier': 'Triple Barrier',
        'tail_sets': 'Tail Sets',
        'matrix_flag': 'Matrix Flag',
        'trend_scanning': 'Trend Scanning',
        'buy_and_hold': 'Buy and Hold',
        "next_period": 'Next Period Labeling'
    }
    
    # Set up the bar positions
    labels = [labeling_method_names.get(label, label) for label in result_df['labeling_method']]
    overall_returns = result_df['overall_return']
    overall_returns_tc = result_df['overall_return_TC']
    
    x = range(len(labels))
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Define colors
    pink_color = '#DA70D6'
    purple_color = '#9932CC'
    black_color = '#000000'
    
    # Create the bars
    bar_width = 0.35
    ax.bar(x, overall_returns * 100, width=bar_width, color=pink_color, label='Accumulated return')
    ax.bar([p + bar_width for p in x], overall_returns_tc * 100, width=bar_width, color=purple_color, label='Accumulated return after transaction costs')
    
    # Add the labels, title and legend
    ax.set_xlabel('Labeling Method')
    ax.set_ylabel('Return in percentage of investment')
    ax.set_title('Overall Returns with and without Transaction Costs')
    ax.set_xticks([p + bar_width/2 for p in x])
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.legend()
    
    # Add a horizontal line at the 0 return mark
    ax.axhline(y=0, color=black_color, linewidth=1)
    
    # Display the plot
    plt.tight_layout()
    
    # Create directory if it does not exist
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.savefig(path, bbox_inches='tight')
    plt.close(fig)

In [5]:
def normalize(series):
    """
    Normalize a pandas Series using min-max normalization.

    Parameters:
    series (pd.Series): The data series to be normalized.

    Returns:
    pd.Series: The normalized data series with values ranging from 0 to 1.
    """
    # Calculate the minimum value of the series
    min_value = series.min()
    
    # Calculate the maximum value of the series
    max_value = series.max()
    
    # Apply the min-max normalization formula
    normalized_series = (series - min_value) / (max_value - min_value)
    
    return normalized_series


In [6]:
def visualize_risk_metrics_radar_plot(result_df, path):
    """
    Visualize the risk metrics for all labeling techniques using a radar plot.

    Parameters:
    result_df (pd.DataFrame): The DataFrame containing 'labeling_method', 'sharpe_ratio', 'sharpe_ratio_TC', 'maximum_drawdown', and 'maximum_drawdown_TC'.
    path (str): The path to save the plot.
    """
    # Define the full English names for the labeling methods
    labeling_method_names = {
        'excess_over_mean': 'Excess over Mean',
        'excess_over_median': 'Excess over Median',
        'fixed_time_horizon': 'Fixed Time Horizon',
        'triple_barrier': 'Triple Barrier',
        'tail_sets': 'Tail Sets',
        'matrix_flag': 'Matrix Flag',
        'trend_scanning': 'Trend Scanning',
        'buy_and_hold': 'Buy and Hold',
        "next_period": 'Next Period Labeling'
    }

    # Check if the DataFrame is empty
    if result_df.empty:
        print("The input DataFrame is empty. Please provide a valid DataFrame.")
        return

    # Ensure that the DataFrame has the required columns
    required_columns = ['labeling_method', 'sharpe_ratio', 'sharpe_ratio_TC', 'maximum_drawdown', 'maximum_drawdown_TC']
    if not all(column in result_df.columns for column in required_columns):
        print(f"DataFrame must contain the following columns: {required_columns}")
        return

    # Handle all-zero maximum drawdown values
    if result_df['maximum_drawdown'].eq(0).all():
        result_df['maximum_drawdown'] = 0.01
    if result_df['maximum_drawdown_TC'].eq(0).all():
        result_df['maximum_drawdown_TC'] = 0.01

    # Normalize the values
    result_df['sharpe_ratio_norm'] = normalize(result_df['sharpe_ratio'])
    result_df['sharpe_ratio_TC_norm'] = normalize(result_df['sharpe_ratio_TC'])
    result_df['maximum_drawdown_norm'] = 1 - normalize(result_df['maximum_drawdown'])  # Inverting to show drawdown as risk
    result_df['maximum_drawdown_TC_norm'] = 1 - normalize(result_df['maximum_drawdown_TC'])  # Inverting to show drawdown as risk

    # Calculate a risk score to sort the methods
    result_df['risk_score'] = result_df[['sharpe_ratio_norm', 'sharpe_ratio_TC_norm', 'maximum_drawdown_norm', 'maximum_drawdown_TC_norm']].sum(axis=1)
    result_df = result_df.sort_values(by='risk_score')

    # Set up the data for the radar plot
    categories = ['Sharpe Ratio', 'Sharpe Ratio TC', 'Maximum Drawdown', 'Maximum Drawdown TC']
    num_vars = len(categories)

    # Create a color palette from dark to light colors
    cmap = get_cmap('viridis')
    colors = [cmap(i / (len(result_df) - 1)) for i in range(len(result_df))]

    # Initialize the radar plot
    fig, ax = plt.subplots(figsize=(12, 12), subplot_kw=dict(polar=True))

    # Helper function to get the normalized values for each labeling method
    def get_normalized_values(label):
        row = result_df[result_df['labeling_method'] == label]
        return [
            row['sharpe_ratio_norm'].values[0],
            row['sharpe_ratio_TC_norm'].values[0],
            row['maximum_drawdown_norm'].values[0],
            row['maximum_drawdown_TC_norm'].values[0]
        ]

    # Plot each labeling method
    for idx, label in enumerate(result_df['labeling_method']):
        values = get_normalized_values(label)
        values += values[:1]  # Ensure the values wrap around to the starting point
        angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
        angles += angles[:1]  # Ensure the angles wrap around to the starting point

        ax.fill(angles, values, color=colors[idx], alpha=0.25)
        ax.plot(angles, values, color=colors[idx], label=labeling_method_names.get(label, label))

    # Add labels to the plot
    ax.set_yticklabels([])
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels([])

    # Move the labels outside the plot
    for angle, label in zip(angles[:-1], categories):
        ax.text(angle, 1.1, label, horizontalalignment='center', size=12, weight='semibold', verticalalignment='top')

    # Create the legend manually
    sorted_labels = [labeling_method_names.get(label, label) for label in result_df['labeling_method']]
    ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))

    # Display the plot
    plt.tight_layout()

    # Create directory if it does not exist
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.savefig(path, bbox_inches='tight')
    plt.close(fig)

In [7]:
def calculate_strategy_returns(df, transaction_cost=0.006):
    """
    Calculate the returns of a trading strategy, including the effect of transaction costs.

    Parameters:
    df (pd.DataFrame): DataFrame containing 'Price' and 'predicted label' columns.
    transaction_cost (float): The transaction cost per trade as a percentage. Default is 0.006 (0.6%).

    Returns:
    pd.DataFrame: Updated DataFrame with additional columns for percentage change, strategy returns,
                  transaction costs, and strategy returns after transaction costs.
    """
    # Calculate the percentage change in price from one row to the next
    df['Pct_Change'] = df['Price'].pct_change().fillna(0)
    
    # Calculate the strategy return by multiplying percentage change by the predicted label
    df['Strategy_Return'] = df['Pct_Change'] * df['predicted label']
    
    # Calculate the transaction cost based on the absolute value of the predicted label
    df['Transaction_Cost'] = df['predicted label'].abs() * transaction_cost
    
    # Calculate the strategy return after transaction costs
    df['Strategy_Return_TC'] = df['Strategy_Return'] - df['Transaction_Cost']
    
    # Set strategy returns to 0 where the predicted label is 0 (no position)
    df.loc[df['predicted label'] == 0, ['Strategy_Return', 'Strategy_Return_TC']] = 0
    
    return df

In [8]:
def plot_distribution_combined(dictionary, path, transaction_cost=0.006):
    """
    Plot the distribution of strategy returns for different labeling methods.

    Parameters:
    dictionary (dict): Dictionary where keys are labeling methods and values are dictionaries of currency data.
                       Each currency data dictionary contains DataFrame with trading data under the key "test".
    path (str): Path to save the plot image.
    transaction_cost (float): The transaction cost per trade as a percentage. Default is 0.006 (0.6%).

    Returns:
    None
    """
    bins = [-np.inf, -0.5, -0.45, -0.4, -0.35, -0.3, -0.25, -0.2, -0.15, -0.1, -0.05, 0, 
            0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, np.inf]
    bin_labels = ["<-50%", "-50% to -45%", "-45% to -40%", "-40% to -35%", "-35% to -30%", 
                  "-30% to -25%", "-25% to -20%", "-20% to -15%", "-15% to -10%", "-10% to -5%", 
                  "-5% to 0%", "0% to 5%", "5% to 10%", "10% to 15%", "15% to 20%", "20% to 25%", 
                  "25% to 30%", "30% to 35%", "35% to 40%", "40% to 45%", "45% to 50%", ">50%"]
    
    labeling_method_names = {
        'excess_over_mean': 'Excess over Mean',
        'excess_over_median': 'Excess over Median',
        'fixed_time_horizon': 'Fixed Time Horizon',
        'triple_barrier': 'Triple Barrier',
        'tail_sets': 'Tail Sets',
        'matrix_flag': 'Matrix Flag',
        'trend_scanning': 'Trend Scanning',
        'buy_and_hold': 'Buy and Hold',
        "next_period": 'Next Period Labeling'
    }
    
    all_counts = pd.DataFrame(0, index=bin_labels, columns=dictionary.keys())
    
    for labeling_method, currency_data in dictionary.items():
        all_trades_df = pd.DataFrame()
        
        for currency, data in currency_data.items():
            strategy_df = calculate_strategy_returns(data["test"], transaction_cost)
            all_trades_df = pd.concat([all_trades_df, strategy_df], ignore_index=True)
        
        all_trades_df['Return_Category'] = pd.cut(all_trades_df['Strategy_Return'], bins=bins, labels=bin_labels)
        counts = all_trades_df['Return_Category'].value_counts().reindex(bin_labels, fill_value=0)
        all_counts[labeling_method] = counts
    
    # Rename the columns to their full English names
    all_counts.rename(columns=labeling_method_names, inplace=True)
    
    # Create a color palette from dark to light colors
    cmap = get_cmap('viridis')
    colors = [cmap(i / (len(all_counts.columns) - 1)) for i in range(len(all_counts.columns))]

    # Main plot
    fig, ax_main = plt.subplots(figsize=(14, 8))

    # Main bar chart
    all_counts.plot(kind='bar', stacked=True, color=colors, ax=ax_main)
    ax_main.set_title('Return Distribution for All Labeling Methods')
    ax_main.set_xlabel('Return Category')
    ax_main.set_ylabel('Number of Trades')
    ax_main.legend(title='Labeling Method')

    # Left tail magnification
    left_tail = all_counts.iloc[:6]  # From -50% to -25%
    ax_left = inset_axes(ax_main, width="45%", height="45%", loc='lower left', 
                         bbox_to_anchor=(0.05, 0.3, 0.45, 0.45), bbox_transform=ax_main.transAxes)
    left_tail.plot(kind='bar', stacked=True, color=colors, ax=ax_left, legend=False)
    ax_left.set_title('Left Tail')
    ax_left.yaxis.set_label_position("right")
    ax_left.yaxis.tick_right()
    ax_left.set_xlabel('')
    ax_left.set_ylabel('')
    ax_left.set_xlim(left=0)

    # Right tail magnification
    right_tail = all_counts.iloc[16:]  # From 20-25%
    ax_right = inset_axes(ax_main, width="45%", height="45%", loc='lower right', 
                          bbox_to_anchor=(0.55, 0.3, 0.45, 0.45), bbox_transform=ax_main.transAxes)
    right_tail.plot(kind='bar', stacked=True, color=colors, ax=ax_right, legend=False)
    ax_right.set_title('Right Tail')
    ax_right.set_xlabel('')
    ax_right.set_ylabel('')

    plt.tight_layout()
    
    # Create directory if it does not exist
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.savefig(path, bbox_inches='tight')
    plt.close(fig)

In [9]:
def plot_distribution_combined(dictionary, path, transaction_cost=0.006):
    """
    Plot the distribution of strategy returns for different labeling methods.

    Parameters:
    dictionary (dict): Dictionary where keys are labeling methods and values are dictionaries of currency data.
                       Each currency data dictionary contains DataFrame with trading data under the key "test".
    path (str): Path to save the plot image.
    transaction_cost (float): The transaction cost per trade as a percentage. Default is 0.006 (0.6%).

    Returns:
    None
    """
    bins = [-np.inf, -0.5, -0.45, -0.4, -0.35, -0.3, -0.25, -0.2, -0.15, -0.1, -0.05, 0, 
            0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, np.inf]
    bin_labels = ["<-50%", "-50% to -45%", "-45% to -40%", "-40% to -35%", "-35% to -30%", 
                  "-30% to -25%", "-25% to -20%", "-20% to -15%", "-15% to -10%", "-10% to -5%", 
                  "-5% to 0%", "0% to 5%", "5% to 10%", "10% to 15%", "15% to 20%", "20% to 25%", 
                  "25% to 30%", "30% to 35%", "35% to 40%", "40% to 45%", "45% to 50%", ">50%"]
    
    labeling_method_names = {
        'excess_over_mean': 'Excess over Mean',
        'excess_over_median': 'Excess over Median',
        'fixed_time_horizon': 'Fixed Time Horizon',
        'triple_barrier': 'Triple Barrier',
        'tail_sets': 'Tail Sets',
        'matrix_flag': 'Matrix Flag',
        'trend_scanning': 'Trend Scanning',
        'buy_and_hold': 'Buy and Hold',
        "next_period": 'Next Period Labeling'
    }
    
    all_counts = pd.DataFrame(0, index=bin_labels, columns=dictionary.keys())
    
    for labeling_method, currency_data in dictionary.items():
        all_trades_df = pd.DataFrame()
        
        for currency, data in currency_data.items():
            # Filter out periods where the predicted label is 0
            strategy_df = data["test"]
            strategy_df = strategy_df[strategy_df['predicted label'] != 0]
            strategy_df['Strategy_Return'] = strategy_df['Pct_Change'] * strategy_df['predicted label'] - transaction_cost
            all_trades_df = pd.concat([all_trades_df, strategy_df], ignore_index=True)
        
        all_trades_df['Return_Category'] = pd.cut(all_trades_df['Strategy_Return'], bins=bins, labels=bin_labels)
        counts = all_trades_df['Return_Category'].value_counts().reindex(bin_labels, fill_value=0)
        all_counts[labeling_method] = counts
    
    # Rename the columns to their full English names
    all_counts.rename(columns=labeling_method_names, inplace=True)
    
    # Create a color palette from dark to light colors
    cmap = get_cmap('viridis')
    colors = [cmap(i / (len(all_counts.columns) - 1)) for i in range(len(all_counts.columns))]

    # Main plot
    fig, ax_main = plt.subplots(figsize=(14, 8))

    # Main bar chart
    all_counts.plot(kind='bar', stacked=True, color=colors, ax=ax_main)
    ax_main.set_title('Return Distribution for All Labeling Methods')
    ax_main.set_xlabel('Return Category')
    ax_main.set_ylabel('Number of Trades')
    ax_main.legend(title='Labeling Method')

    # Left tail magnification
    left_tail = all_counts.iloc[:6]  # From -50% to -25%
    ax_left = inset_axes(ax_main, width="45%", height="45%", loc='lower left', 
                         bbox_to_anchor=(0.05, 0.3, 0.45, 0.45), bbox_transform=ax_main.transAxes)
    left_tail.plot(kind='bar', stacked=True, color=colors, ax=ax_left, legend=False)
    ax_left.set_title('Left Tail')
    ax_left.yaxis.set_label_position("right")
    ax_left.yaxis.tick_right()
    ax_left.set_xlabel('')
    ax_left.set_ylabel('')
    ax_left.set_xlim(left=0)

    # Right tail magnification
    right_tail = all_counts.iloc[16:]  # From 20-25%
    ax_right = inset_axes(ax_main, width="45%", height="45%", loc='lower right', 
                          bbox_to_anchor=(0.55, 0.3, 0.45, 0.45), bbox_transform=ax_main.transAxes)
    right_tail.plot(kind='bar', stacked=True, color=colors, ax=ax_right, legend=False)
    ax_right.set_title('Right Tail')
    ax_right.set_xlabel('')
    ax_right.set_ylabel('')

    plt.tight_layout()
    
    # Create directory if it does not exist
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.savefig(path, bbox_inches='tight')
    plt.close(fig)

In [10]:
def plot_label_distribution(df, path):
    """
    This function plots a bar chart showing the distribution of labels for each labeling method.
    
    Parameters:
    df (DataFrame): The DataFrame containing the counts of each label (-1, 0, 1) for different labeling methods.
    
    Returns:
    None: This function displays the bar chart.
    """
    # Calculate the relative distribution
    df_relative = df.div(df.sum(axis=1), axis=0) * 100

    # Define the colors for the labels
    colors = ['#AD27C5', '#0BE5E2', '#52A94B']  # Purple for Short signal, Cyan for No trade signal, and Lime Green for Long signal

    # Plot the bar chart
    ax = df_relative.plot(kind='bar', stacked=True, figsize=(12, 8), color=colors)
    plt.title('Label Distribution by Labeling Method')
    plt.xlabel('Labeling Method')
    plt.ylabel('Percentage')
    plt.legend(['Short signal', 'No trade signal', 'Long signal'], title='Label', loc='upper right', bbox_to_anchor=(1.15, 1.0))
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    # Create directory if it does not exist
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.savefig(path, bbox_inches='tight')
    plt.close()

In [11]:
def visualize_overall_returns_vertical(daily_df, weekly_df, path):
    """
    Visualize the overall returns with and without transaction costs for each labeling method
    for both daily and weekly results.

    Parameters:
    - daily_df (pd.DataFrame): DataFrame containing daily overall returns.
    - weekly_df (pd.DataFrame): DataFrame containing weekly overall returns.
    - path (str): Path to save the plot.
    """
    # Define the full English names for the labeling methods
    labeling_method_names = {
        'excess_over_mean': 'Excess over Mean',
        'excess_over_median': 'Excess over Median',
        'fixed_time_horizon': 'Fixed Time Horizon',
        'triple_barrier': 'Triple Barrier',
        'tail_sets': 'Tail Sets',
        'matrix_flag': 'Matrix Flag',
        'trend_scanning': 'Trend Scanning',
        'buy_and_hold': 'Buy and Hold',
        "next_period": 'Next Period Labeling'
    }

    # Set up the bar positions
    labels = [labeling_method_names.get(label, label) for label in daily_df['labeling_method']]
    
    x = range(len(labels))

    fig, ax = plt.subplots(figsize=(12, 8))

    # Define colors
    pink_color = '#DA70D6'
    pink_dark_color = '#9932CC'
    blue_color = '#1E90FF'
    blue_dark_color = '#104E8B'
    black_color = '#000000'
    
    # Create the bars
    bar_width = 0.2

    ax.barh(x, daily_df['overall_return'] * 100, height=bar_width, color=pink_color, label='Daily accumulated return')
    ax.barh([p + bar_width for p in x], daily_df['overall_return_TC'] * 100, height=bar_width, color=pink_dark_color, label='Daily accumulated return after TC')
    ax.barh([p + 2 * bar_width for p in x], weekly_df['overall_return'] * 100, height=bar_width, color=blue_color, label='Weekly accumulated return')
    ax.barh([p + 3 * bar_width for p in x], weekly_df['overall_return_TC'] * 100, height=bar_width, color=blue_dark_color, label='Weekly accumulated return after TC')

    # Add the labels, title and legend
    ax.set_xlabel('Return in percentage of investment')
    ax.set_ylabel('Labeling Method')
    ax.set_title('Overall Returns with and without Transaction Costs (Daily vs Weekly)')
    ax.set_yticks([p + 1.5 * bar_width for p in x])
    ax.set_yticklabels(labels, rotation=0, ha='right')
    ax.legend()

    # Add a vertical line at the 0 return mark
    ax.axvline(x=0, color=black_color, linewidth=1)

    # Display the plot
    plt.tight_layout()

    # Create directory if it does not exist
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.savefig(path, bbox_inches='tight')
    plt.close(fig)